In [1]:
import os
import sys
import dimod
import matplotlib.pyplot as plt
import numpy as np

this_folder = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.insert(0, this_folder)

from utils import *
from optimized_qubos import *

dim = 2
initial_tensor = get_standard_tensor(dim) #% 2
suggested_optimal = 7 #Proved optimal for multiplying 2x2 matrices
origo = np.tensordot([0]*4, np.tensordot([0]*4, [0]*4, axes=0), axes=0)

In [2]:
# These tensors describe the provably optimal way to multiply 2x2 matrices
strassen_tensors_np = [np.tensordot([0,0,0,1], np.tensordot([-1,0,1,0], [1,0,1,0], axes=0), axes=0),
           np.tensordot([1,1,0,0], np.tensordot([0,0,0,1], [-1,1,0,0], axes=0), axes=0),
           np.tensordot([-1,0,1,0], np.tensordot([1,1,0,0], [0,0,0,1], axes=0), axes=0),
           np.tensordot([1,0,0,1], np.tensordot([1,0,0,1], [1,0,0,1], axes=0), axes=0),
           np.tensordot([0,1,0,-1], np.tensordot([0,0,1,1], [1,0,0,0], axes=0), axes=0),
           np.tensordot([1,0,0,0], np.tensordot([0,1,0,-1], [0,1,0,1], axes=0), axes=0),
           np.tensordot([0,0,1,1], np.tensordot([1,0,0,0], [0,0,1,-1], axes=0), axes=0)]

In [3]:
def construct_all_tensors(sample, dim):
    print(sample)
    positive_linear_vars = []
    tensors = []
    for i in range(suggested_optimal):
        x, y, z = [], [], []
        for j in range(dim**2):
            x.append(sample[str(i) + "x" + str(j)])
            y.append(sample[str(i) + "y" + str(j)])
            z.append(sample[str(i) + "z" + str(j)])
        tensors.append([x,y,z])
    return tensors

In [4]:
weight = 15
initial_tensor = get_standard_tensor(dim)
bqm, hubo, aux_ids = get_full_higher_order_binary_optimization_problem(initial_tensor, dim, suggested_optimal, weight)

print("Number of linear variables: ", len(bqm.linear))
print("Number of quadratic variables: ", len(bqm.quadratic))

sample, energy, sampleset = solve_bqm_in_leap(bqm, "LeapHybrid")

tensors = construct_all_tensors(sample, dim)

initial_tensor = get_standard_tensor(dim)
for t in tensors:
    print(t)
    initial_tensor = np.mod(initial_tensor - np.tensordot(t[0], np.tensordot(t[1], t[2], axes=0), axes=0), 2)

# After substraction, the tensor should be the origo tensor (all entries are 0)
if np.count_nonzero(initial_tensor.flatten()) == 0:
    print("Success!")
else:
    print("No success")
    
#print(tensors)
print(initial_tensor)

[('0x0', '0y0', '0z0'), ('1x0', '1y0', '1z0'), ('2x0', '2y0', '2z0'), ('3x0', '3y0', '3z0'), ('4x0', '4y0', '4z0'), ('5x0', '5y0', '5z0'), ('6x0', '6y0', '6z0')]
[('0x0', '0y0', '0z1'), ('1x0', '1y0', '1z1'), ('2x0', '2y0', '2z1'), ('3x0', '3y0', '3z1'), ('4x0', '4y0', '4z1'), ('5x0', '5y0', '5z1'), ('6x0', '6y0', '6z1')]
[('0x0', '0y0', '0z2'), ('1x0', '1y0', '1z2'), ('2x0', '2y0', '2z2'), ('3x0', '3y0', '3z2'), ('4x0', '4y0', '4z2'), ('5x0', '5y0', '5z2'), ('6x0', '6y0', '6z2')]
[('0x0', '0y0', '0z3'), ('1x0', '1y0', '1z3'), ('2x0', '2y0', '2z3'), ('3x0', '3y0', '3z3'), ('4x0', '4y0', '4z3'), ('5x0', '5y0', '5z3'), ('6x0', '6y0', '6z3')]
[('0x0', '0y1', '0z0'), ('1x0', '1y1', '1z0'), ('2x0', '2y1', '2z0'), ('3x0', '3y1', '3z0'), ('4x0', '4y1', '4z0'), ('5x0', '5y1', '5z0'), ('6x0', '6y1', '6z0')]
[('0x0', '0y1', '0z1'), ('1x0', '1y1', '1z1'), ('2x0', '2y1', '2z1'), ('3x0', '3y1', '3z1'), ('4x0', '4y1', '4z1'), ('5x0', '5y1', '5z1'), ('6x0', '6y1', '6z1')]
[('0x0', '0y1', '0z2'), ('1x

The following line shows that the quantum annealer did not find the optimal point because we can construct a point with lower energy.

In [5]:
bqm.energy(get_test_strassen_test_tensor(bqm, aux_ids))

40.0